# MapLibre stability check: random-walk marker (Copenhagen City Hall)

This notebook stress-tests **anymap-ts** (MapLibre backend via **anywidget**) by animating a marker along a precomputed random-walk path around Copenhagen City Hall (Rådhuspladsen).

Implementation detail: it uses `Map.animate_along_route(...)`, which performs smooth in-JS marker updates (`setLngLat`) rather than repeatedly adding/removing markers.

In [ ]:
from __future__ import annotations

import math
import random
from typing import List, Tuple
from anymap_ts import Map


In [ ]:
#This function generates a random walk around a central point (like a city hall) and returns a list of longitude and latitude coordinates. The walk is bounded by a maximum radius, and the step size can be adjusted. The random seed ensures reproducibility of the generated coordinates.

def random_walk_coords(
    center_lnglat: Tuple[float, float],
    *,
    n_points: int = 600,
    step_m: float = 6.0,
    max_radius_m: float = 220.0,
    seed: int = 7,
) -> List[List[float]]:
    """Generate a bounded random walk as a list of [lng, lat] points."""
    rng = random.Random(seed)
    lng0, lat0 = center_lnglat

    # Local tangent-plane approximation (good enough at city scale).
    meters_per_deg_lat = 111_320.0
    meters_per_deg_lng = 111_320.0 * math.cos(math.radians(lat0))

    # Track offsets in meters from the center.
    x_m = 0.0  # east
    y_m = 0.0  # north

    coords: List[List[float]] = [[lng0, lat0]]
    for _ in range(n_points - 1):
        theta = rng.random() * 2.0 * math.pi
        x_m += step_m * math.cos(theta)
        y_m += step_m * math.sin(theta)

        # Soft-bounds: if we drift outside the radius, pull back toward center.
        r = math.hypot(x_m, y_m)
        if r > max_radius_m:
            scale = max_radius_m / r
            x_m *= scale
            y_m *= scale

        lng = lng0 + (x_m / meters_per_deg_lng)
        lat = lat0 + (y_m / meters_per_deg_lat)
        coords.append([lng, lat])

    return coords


# Copenhagen City Hall (approx).
CITY_HALL_LNGLAT = (12.5683, 55.6761)

coords = random_walk_coords(
    CITY_HALL_LNGLAT,
    n_points=900,
    step_m=7.0,
    max_radius_m=250.0,
    seed=42,
)

In [ ]:


m = Map(center=CITY_HALL_LNGLAT, zoom=16.5, height="650px")

# Animate in the frontend (smooth marker updates, no Python loop).
m.animate_along_route(
    coords,
    anim_id="cityhall-walk",
    duration=45_000,
    loop=True,
    marker_color="#3388ff",
    marker_size=1.0,
    show_trail=False,
)

display(m)